In [2]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import re

In [2]:
changes_train_pkl = pd.read_pickle('../../data/jitfine/changes_train.pkl')

In [3]:
print(type(changes_train_pkl), len(changes_train_pkl))

<class 'list'> 4


In [4]:
print('数据量大小：', len(changes_train_pkl[0]),len(changes_train_pkl[1]),
      len(changes_train_pkl[2]),len(changes_train_pkl[3]))

数据量大小： 16374 16374 16374 16374


In [5]:
print('这是commit_hash')
print(changes_train_pkl[0][:5])

这是commit_hash
['2a5a07fcb1a24fded957d260f9a9df988323db19', '4642f1df280891c2238e74935239096fba508ba0', '42ffb77e341635adbf1f2b71727cb586fe2ff672', 'b92f81e2f692f52943c49c595d087d3e3c6c622d', '36a1784d52126394afc5963d8139f8933bdd60c9']


In [6]:
print('这是is_buggy_commit')
print(changes_train_pkl[1][:5])


这是is_buggy_commit
[1.0, 1.0, 1.0, 1.0, 1.0]


In [7]:
print('这是commit_message')
print(changes_train_pkl[2][:5])


这是commit_message
['publish task now uses srcivypattern for ivy files finding for delivery', 'added ivy task : cachefileset', 'add cachefileset task', 'new useRemoteConfig on conf tag in ivyconf . xml , tells to use remote configuration file for repo config', 'new changing attribute on dependency']


In [26]:
print('这是 added_code 和 removed_code')
info = changes_train_pkl[3][2]
info

这是 added_code 和 removed_code


{'added_code': {'Artifact [ ] artifacts = parser . getArtifacts ( new ModuleId ( organisation , module ) , confs [ i ] , cache ) ;',
  'Artifact artifact = ( Artifact ) iter . next ( ) ;',
  'Collection all = new LinkedHashSet ( ) ;',
  'FileSet fileset = new FileSet ( ) ;',
  'Ivy ivy = getIvyInstance ( ) ;',
  'NameEntry ne = fileset . createInclude ( ) ;',
  'String [ ] confs = splitConfs ( conf ) ;',
  'XmlReportParser parser = new XmlReportParser ( ) ;',
  'all . addAll ( Arrays . asList ( artifacts ) ) ;',
  'cache = cache ;',
  'cache = ivy . getDefaultCache ( ) ;',
  'conf = conf ;',
  'conf = getProperty ( conf , ivy , "ivy . resolved . configurations" ) ;',
  'conf = getProperty ( ivy , "ivy . resolved . configurations" ) ;',
  'ensureResolved ( isHaltonfailure ( ) ) ;',
  'fileset . setDir ( getCache ( ) ) ;',
  'fileset . setProject ( getProject ( ) ) ;',
  'for ( Iterator iter = all . iterator ( ) ; iter . hasNext ( ) ; ) {',
  'for ( int i = 0 ; i < confs . length ; i + +

In [27]:
add_code_lines = info['added_code']
delete_code_lines = info['removed_code']
print(type(add_code_lines))
len(add_code_lines)+len(delete_code_lines)

<class 'set'>


84

In [29]:
# add_code_lines






In [3]:
buggy_commit_lines_df = pd.read_pickle('./train_buggy_commit_lines_df.pkl')
buggy_commit_lines_df.info()
buggy_commit_lines_df.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117818 entries, 0 to 117817
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   project      117818 non-null  object
 1   file         117818 non-null  object
 2   change type  117818 non-null  object
 3   commit hash  117818 non-null  object
 4   idx          117818 non-null  int64 
 5   code line    117818 non-null  object
 6   label        117818 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 6.3+ MB


,project,file,change type,commit hash,idx,code line,label
0,ant-ivy,src/java/fr/jayasoft/ivy/ant/IvyPublish.java,added,2a5a07fcb1a24fded957d260f9a9df988323db19,0,"File ivyFile = new File(_cache, IvyPatternHelp...",1
1,ant-ivy,src/java/fr/jayasoft/ivy/ant/IvyPublish.java,added,2a5a07fcb1a24fded957d260f9a9df988323db19,1,},0
2,ant-ivy,src/java/fr/jayasoft/ivy/ant/IvyPublish.java,added,2a5a07fcb1a24fded957d260f9a9df988323db19,2,_srcivypattern = destivypattern;,0


In [4]:
def remove_punctuation(line):
    rule = re.compile(r"[^a-zA-Z0-9\u4e00-\u9fa5]")
    line = rule.sub('',line)
    return line

for i in tqdm(range(len(buggy_commit_lines_df))):
    x = buggy_commit_lines_df['code line'][i]
    try:
        remove_punctuation(x)
    except TypeError:
        print(x)

100%|██████████| 117818/117818 [00:00<00:00, 132677.64it/s]


In [ ]:
# commit_id = '2a5a07fcb1a24fded957d260f9a9df988323db19'
commit_id = '42ffb77e341635adbf1f2b71727cb586fe2ff672'

In [43]:
if commit_id in buggy_commit_lines_df['commit hash'].to_list():
    commit_info_df = buggy_commit_lines_df[buggy_commit_lines_df['commit hash']==commit_id]
    commit_info_df = commit_info_df.reset_index(drop=True)

    import re
    def remove_punctuation(line):
        rule = re.compile(r"[^a-zA-Z0-9\u4e00-\u9fa5]")
        line = rule.sub('',line)
        return line


    commit_info_df['code line'] = commit_info_df['code line'].apply(lambda x: remove_punctuation(x))

    add_code_lines_dict = {remove_punctuation(line): line for line in add_code_lines}
    delete_code_lines_dict = {remove_punctuation(line): line for line in delete_code_lines}


    commit_info_df_added = commit_info_df[commit_info_df['change type']=='added']
    commit_info_df_added  = commit_info_df_added.reset_index(drop=True)
    add_code_lines = []
    add_code_lines_labels = []
    for i in range(len(commit_info_df_added)):
        if commit_info_df_added['code line'][i] in add_code_lines_dict.keys():
            add_code_lines.append(add_code_lines_dict[commit_info_df_added['code line'][i]])
            add_code_lines_labels.append(commit_info_df_added['label'][i])

        else:
            print('added lines matching exists wrong')



    commit_info_df_deleted = commit_info_df[commit_info_df['change type']=='deleted']
    commit_info_df_deleted  = commit_info_df_deleted.reset_index(drop=True)
    delete_code_lines = []
    delete_code_lines_labels = []
    for i in range(len(commit_info_df_deleted)):
        if commit_info_df_deleted['code line'][i] in delete_code_lines_dict.keys():
            delete_code_lines.append(delete_code_lines_dict[commit_info_df_deleted['code line'][i]])
            delete_code_lines_labels.append(commit_info_df_deleted['label'][i])
        else:
            print('added lines matching exists wrong')
else:
    add_code_lines_labels = [0]*len(add_code_lines)
    delete_code_lines_labels = [0]*len(delete_code_lines)

True